In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
import xlsxwriter
import unicodedata
import os

# Load datasets
print("Loading datasets...")
final_data = pd.read_excel('Final_Claims_Data_with_AggregatesV2.xlsx', dtype={'Policy': str, 'Unit': str})
insurance_data = pd.read_csv('Transformed_Auto_Insurance_Cleaned_For_GLM_V2.csv', encoding='utf-8-sig', dtype={'Policy': str, 'Unit': str})

# Deduplication
print("\U0001f9f9 Removing duplicate insurance rows...")
insurance_data = insurance_data.drop_duplicates()
insurance_data = insurance_data.drop_duplicates(subset=['Policy', 'Unit', 'Start_Date', 'End_Date'])
insurance_data = insurance_data[insurance_data['PrimaSuscrita'] > 0]
insurance_data.to_csv('InsuranceData_Filtered_Deduplicated.csv', index=False, encoding='utf-8-sig')

# Convert dates
print("\ud83d\uddd3\ufe0f Converting date fields...")
final_data['Start_Date'] = pd.to_datetime(final_data['Start_Date'], errors='coerce')
final_data['Occurrence_date'] = pd.to_datetime(final_data['Occurrence_date'], errors='coerce')
insurance_data['Start_Date'] = pd.to_datetime(insurance_data['Start_Date'], errors='coerce')
insurance_data['End_Date'] = pd.to_datetime(insurance_data['End_Date'], errors='coerce')

# Create Key
final_data['Key'] = final_data['Policy'] + '_' + final_data['Unit']
insurance_data['Key'] = insurance_data['Policy'] + '_' + insurance_data['Unit']

# Format insurance_data
print("\ud83e\uddfc Formatting insurance_data...")
for col in tqdm(insurance_data.columns, desc="Formatting insurance_data"):
    if insurance_data[col].dtype == "object":
        insurance_data[col] = insurance_data[col].fillna("").astype(str)
    elif pd.api.types.is_numeric_dtype(insurance_data[col]):
        insurance_data[col] = pd.to_numeric(insurance_data[col], errors='coerce').fillna(0)

# Unique index for claims
final_data = final_data.reset_index().rename(columns={'index': 'Claim_Row_ID'})

# Filter insurance keys found in claims
print("\ud83d\udd0e Filtering relevant policy-units...")
matching_keys = final_data['Key'].unique()
filtered_insurance_data = insurance_data[insurance_data['Key'].isin(matching_keys)]
non_matching_rows = insurance_data[~insurance_data['Key'].isin(matching_keys)]

# 1-to-many Matching
aggregated_results = []
used_claim_ids = set()
used_insurance_indices = set()
unmatched_claims = []

print("\ud83d\udccc Matching claims to insurance rows (all that match)...")
for idx, ins_row in tqdm(filtered_insurance_data.iterrows(), total=len(filtered_insurance_data), desc="Matching multiple claims"):
    if idx in used_insurance_indices:
        continue

    policy_unit_key = ins_row['Key']
    start_date = ins_row['Start_Date']
    end_date = ins_row['End_Date']

    potential_claims = final_data[
        (final_data['Key'] == policy_unit_key) &
        (~final_data['Claim_Row_ID'].isin(used_claim_ids))
    ]

    if potential_claims.empty:
        unmatched_claims.append({**ins_row[['Policy', 'Unit', 'Start_Date', 'End_Date']].to_dict(),
                                 'Reason': 'No available unmatched claims for this Policy-Unit'})
        continue

    matching_claims = potential_claims[potential_claims['Start_Date'] == start_date]

    if matching_claims.empty:
        matching_claims = potential_claims[
            (potential_claims['Occurrence_date'] >= start_date) &
            (potential_claims['Occurrence_date'] <= end_date)
        ]

    if matching_claims.empty:
        matching_claims = potential_claims[
            (potential_claims['Occurrence_date'] >= start_date - timedelta(days=365)) &
            (potential_claims['Occurrence_date'] <= end_date + timedelta(days=365))
        ]

    if not matching_claims.empty:
        selected_claim = matching_claims.copy()
        used_claim_ids.update(selected_claim['Claim_Row_ID'].values)
        used_insurance_indices.add(idx)

        aggregated_data = selected_claim.groupby(['Policy', 'Unit', 'Coverage_Type']).agg({
            'Payed_Amount': 'sum',
            'Last_Pending_Amount': 'sum',
            'Incurred': 'sum',
            'Claim_Number': pd.Series.nunique
        }).reset_index()

        aggregated_data['Policy'] = ins_row['Policy']
        aggregated_data['Unit'] = ins_row['Unit']
        aggregated_data['Start_Date'] = start_date
        aggregated_data['End_Date'] = end_date
        aggregated_data['Insurance_Row_Index'] = idx

        aggregated_results.append(aggregated_data)
    else:
        unmatched_claims.append({**ins_row[['Policy', 'Unit', 'Start_Date', 'End_Date']].to_dict(),
                                 'Reason': 'No claim matched this period'})

# Aggregation and export
if aggregated_results:
    print("\ud83d\udcca Aggregating results...")
    aggregated_df = pd.concat(aggregated_results)
    pivot_df = aggregated_df.pivot_table(
        index='Insurance_Row_Index',
        columns='Coverage_Type',
        values=['Payed_Amount', 'Last_Pending_Amount', 'Incurred', 'Claim_Number'],
        aggfunc='sum'
    ).fillna(0)
    pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]
    pivot_df.reset_index(inplace=True)

    insurance_data = insurance_data.reset_index().rename(columns={'index': 'Insurance_Row_Index'})
    final_result = insurance_data.merge(pivot_df, on='Insurance_Row_Index', how='left')

    value_cols = [col for col in final_result.columns if col.startswith(('Claim_Number_', 'Payed_Amount_', 'Last_Pending_Amount_', 'Incurred_'))]
    final_result[value_cols] = final_result[value_cols].fillna(0)

    final_result['Total_Claim_Count'] = final_result.filter(like='Claim_Number_').sum(axis=1)
    final_result['Total_Payed_Amount'] = final_result.filter(like='Payed_Amount_').sum(axis=1)
    final_result['Total_Pending_Amount'] = final_result.filter(like='Last_Pending_Amount_').sum(axis=1)
    final_result['Total_Incurred_Amount'] = final_result.filter(like='Incurred_').sum(axis=1)

    final_result.drop(columns=['Insurance_Row_Index', 'Key', 'Cliente'], errors='ignore', inplace=True)

    # Unit Count per Policy-Year
    print("\ud83d\udd39 Calculating Unit_Count_Per_Policy_Year...")
    final_result['Start_Year'] = pd.to_datetime(final_result['Start_Date']).dt.year
    unit_counts = final_result.groupby(['Policy', 'Start_Year'])['Unit'].nunique().reset_index()
    unit_counts = unit_counts.rename(columns={'Unit': 'Unit_Count_Per_Policy_Year'})
    final_result = final_result.merge(unit_counts, on=['Policy', 'Start_Year'], how='left')

    # Historical metrics
    print("\ud83d\udcca Calculating historical metrics...")
    final_result['PU_Key'] = final_result['Policy'] + '_' + final_result['Unit']
    with_claims = final_result[(final_result['Total_Claim_Count'] > 0) | (final_result['Total_Incurred_Amount'] > 0)]
    claim_count_results = []

    grouped = with_claims.groupby('PU_Key')
    for key, group in tqdm(grouped, desc="Historical calculations", total=len(grouped)):
        group = group.sort_values('Start_Date')
        hist_claims = []
        hist_ratios = []
        for i in range(len(group)):
            past = group[group['Start_Date'] < group.iloc[i]['Start_Date']]
            hist_claims.append(past['Total_Claim_Count'].sum())
            premium = past['PrimaSuscrita'].sum()
            incurred = past['Total_Incurred_Amount'].sum()
            hist_ratios.append(incurred / premium if premium else None)
        group['Historical_Claim_Count'] = hist_claims
        group['Historical_Loss_Ratio'] = hist_ratios
        claim_count_results.append(group)

    enriched = pd.concat(claim_count_results, ignore_index=True)
    enriched['Current_Loss_Ratio'] = enriched['Total_Incurred_Amount'] / enriched['PrimaSuscrita']

    final_result = final_result.drop(columns=['PU_Key'], errors='ignore')
    final_result = final_result.merge(
        enriched[['Policy', 'Unit', 'Start_Date', 'Historical_Claim_Count', 'Historical_Loss_Ratio', 'Current_Loss_Ratio']],
        on=['Policy', 'Unit', 'Start_Date'],
        how='left'
    )

    print("\ud83e\uddfc Final formatting for export...")
    for col in tqdm(final_result.columns, desc="Final format"):
        if final_result[col].dtype == "object":
            final_result[col] = final_result[col].fillna("").astype(str)
        elif pd.api.types.is_numeric_dtype(final_result[col]):
            final_result[col] = pd.to_numeric(final_result[col], errors='coerce').fillna(0)

    # Export
    print("\ud83d\udcc5 Exporting final results...")
    final_result.to_csv('Insurance_1to1_Merged_With_Loss_Ratio.csv', index=False, encoding='utf-8-sig')
    final_result.to_parquet("Insurance_1to1_Merged_With_Loss_Ratio.parquet", index=False)
    with pd.ExcelWriter("Insurance_1to1_Merged_With_Loss_Ratio.xlsx", engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        final_result.to_excel(writer, index=False, sheet_name='Sheet1')

    # Optional: Remove accents
    def remove_accents(text):
        if isinstance(text, str):
            text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
            text = text.replace('ñ', 'n').replace('Ñ', 'N')
        return text

    print("\ud83c\udf10 Removing accents from string columns...")
    for col in tqdm(final_result.columns, desc="Removing accents"):
        if final_result[col].dtype == "object":
            final_result[col] = final_result[col].apply(remove_accents)

    final_result.to_csv('Insurance_1to1_Merged_With_Loss_Ratio_CleanText.csv', index=False, encoding='utf-8-sig')
else:
    final_result = pd.DataFrame()
    print("\u26a0\ufe0f No matched claims found.")

# Export unmatched claims
unmatched_all = unmatched_claims + [
    {
        'Policy': row['Policy'],
        'Unit': row['Unit'],
        'Start_Date': row['Start_Date'],
        'End_Date': row['End_Date'],
        'Reason': 'Policy-Unit not found in claims data'
    }
    for _, row in non_matching_rows.iterrows()
]
if unmatched_all:
    unmatched_df = pd.DataFrame(unmatched_all)
    unmatched_df.to_csv('Unmatched_Claims_Final.csv', index=False, encoding='utf-8-sig')
    print(f"\u26a0\ufe0f Exported {len(unmatched_df)} unmatched entries.")
else:
    print("\u2705 All insurance rows matched successfully.")

# Summary
print(f"\n\ud83d\udcb0 Total Incurred Matched: {final_result['Total_Incurred_Amount'].sum():,.2f}")
print(f"\ud83d\udd52 Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
import unicodedata
import os

# Load datasets
print("Loading datasets...")
final_data = pd.read_excel('Final_Claims_Data_with_AggregatesV2.xlsx', dtype={'Policy': str, 'Unit': str})
insurance_data = pd.read_csv('Transformed_Auto_Insurance_Cleaned_For_GLM_V2.csv', encoding='utf-8-sig', dtype={'Policy': str, 'Unit': str})

# Deduplication
print("🧹 Removing duplicate insurance rows...")
insurance_data = insurance_data.drop_duplicates()
insurance_data = insurance_data.drop_duplicates(subset=['Policy', 'Unit', 'Start_Date', 'End_Date'])
insurance_data = insurance_data[insurance_data['PrimaSuscrita'] > 0]
insurance_data.to_csv('InsuranceData_Filtered_Deduplicated.csv', index=False, encoding='utf-8-sig')

# Convert dates
print("📅 Converting date fields...")
final_data['Start_Date'] = pd.to_datetime(final_data['Start_Date'], errors='coerce')
final_data['Occurrence_date'] = pd.to_datetime(final_data['Occurrence_date'], errors='coerce')
insurance_data['Start_Date'] = pd.to_datetime(insurance_data['Start_Date'], errors='coerce')
insurance_data['End_Date'] = pd.to_datetime(insurance_data['End_Date'], errors='coerce')

# Create Key
final_data['Key'] = final_data['Policy'] + '_' + final_data['Unit']
insurance_data['Key'] = insurance_data['Policy'] + '_' + insurance_data['Unit']

# Format insurance_data
print("🧼 Formatting insurance_data...")
for col in tqdm(insurance_data.columns, desc="Formatting insurance_data"):
    if insurance_data[col].dtype == "object":
        insurance_data[col] = insurance_data[col].fillna("").astype(str)
    elif pd.api.types.is_numeric_dtype(insurance_data[col]):
        insurance_data[col] = pd.to_numeric(insurance_data[col], errors='coerce').fillna(0)

# Unique index for claims
final_data = final_data.reset_index().rename(columns={'index': 'Claim_Row_ID'})

# Filter insurance keys found in claims
print("🔎 Filtering relevant policy-units...")
matching_keys = final_data['Key'].unique()
filtered_insurance_data = insurance_data[insurance_data['Key'].isin(matching_keys)]
non_matching_rows = insurance_data[~insurance_data['Key'].isin(matching_keys)]

# 1-to-many Matching
aggregated_results = []
used_claim_ids = set()
used_insurance_indices = set()
unmatched_claims = []

print("📌 Matching claims to insurance rows (all that match)...")
for idx, ins_row in tqdm(filtered_insurance_data.iterrows(), total=len(filtered_insurance_data), desc="Matching multiple claims"):
    if idx in used_insurance_indices:
        continue

    policy_unit_key = ins_row['Key']
    start_date = ins_row['Start_Date']
    end_date = ins_row['End_Date']

    potential_claims = final_data[
        (final_data['Key'] == policy_unit_key) &
        (~final_data['Claim_Row_ID'].isin(used_claim_ids))
    ]

    if potential_claims.empty:
        unmatched_claims.append({**ins_row[['Policy', 'Unit', 'Start_Date', 'End_Date']].to_dict(),
                                 'Reason': 'No available unmatched claims for this Policy-Unit'})
        continue

    matching_claims = potential_claims[potential_claims['Start_Date'] == start_date]

    if matching_claims.empty:
        matching_claims = potential_claims[
            (potential_claims['Occurrence_date'] >= start_date) &
            (potential_claims['Occurrence_date'] <= end_date)
        ]

    if matching_claims.empty:
        matching_claims = potential_claims[
            (potential_claims['Occurrence_date'] >= start_date - timedelta(days=365)) &
            (potential_claims['Occurrence_date'] <= end_date + timedelta(days=365))
        ]

    if not matching_claims.empty:
        selected_claim = matching_claims.copy()
        used_claim_ids.update(selected_claim['Claim_Row_ID'].values)
        used_insurance_indices.add(idx)

        aggregated_data = selected_claim.groupby(['Policy', 'Unit', 'Coverage_Type']).agg({
            'Payed_Amount': 'sum',
            'Last_Pending_Amount': 'sum',
            'Incurred': 'sum',
            'Claim_Number': pd.Series.nunique
        }).reset_index()

        aggregated_data['Policy'] = ins_row['Policy']
        aggregated_data['Unit'] = ins_row['Unit']
        aggregated_data['Start_Date'] = start_date
        aggregated_data['End_Date'] = end_date
        aggregated_data['Insurance_Row_Index'] = idx

        aggregated_results.append(aggregated_data)
    else:
        unmatched_claims.append({**ins_row[['Policy', 'Unit', 'Start_Date', 'End_Date']].to_dict(),
                                 'Reason': 'No claim matched this period'})

# Aggregation and export
if aggregated_results:
    print("📊 Aggregating results...")
    aggregated_df = pd.concat(aggregated_results)
    pivot_df = aggregated_df.pivot_table(
        index='Insurance_Row_Index',
        columns='Coverage_Type',
        values=['Payed_Amount', 'Last_Pending_Amount', 'Incurred', 'Claim_Number'],
        aggfunc='sum'
    ).fillna(0)
    pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]
    pivot_df.reset_index(inplace=True)

    insurance_data = insurance_data.reset_index().rename(columns={'index': 'Insurance_Row_Index'})
    final_result = insurance_data.merge(pivot_df, on='Insurance_Row_Index', how='left')

    value_cols = [col for col in final_result.columns if col.startswith(('Claim_Number_', 'Payed_Amount_', 'Last_Pending_Amount_', 'Incurred_'))]
    final_result[value_cols] = final_result[value_cols].fillna(0)

    final_result['Total_Claim_Count'] = final_result.filter(like='Claim_Number_').sum(axis=1)
    final_result['Total_Payed_Amount'] = final_result.filter(like='Payed_Amount_').sum(axis=1)
    final_result['Total_Pending_Amount'] = final_result.filter(like='Last_Pending_Amount_').sum(axis=1)
    final_result['Total_Incurred_Amount'] = final_result.filter(like='Incurred_').sum(axis=1)

    final_result.drop(columns=['Insurance_Row_Index', 'Key', 'Cliente'], errors='ignore', inplace=True)

    # Unit Count per Policy-Year
    print("🔢 Calculating Unit_Count_Per_Policy_Year...")
    final_result['Start_Year'] = pd.to_datetime(final_result['Start_Date']).dt.year
    unit_counts = final_result.groupby(['Policy', 'Start_Year'])['Unit'].nunique().reset_index()
    unit_counts = unit_counts.rename(columns={'Unit': 'Unit_Count_Per_Policy_Year'})
    final_result = final_result.merge(unit_counts, on=['Policy', 'Start_Year'], how='left')

    # Historical metrics
    print("📊 Calculating historical metrics...")
    final_result['PU_Key'] = final_result['Policy'] + '_' + final_result['Unit']
    with_claims = final_result[(final_result['Total_Claim_Count'] > 0) | (final_result['Total_Incurred_Amount'] > 0)]
    claim_count_results = []

    grouped = with_claims.groupby('PU_Key')
    for key, group in tqdm(grouped, desc="Historical calculations", total=len(grouped)):
        group = group.sort_values('Start_Date')
        hist_claims = []
        hist_ratios = []
        for i in range(len(group)):
            past = group[group['Start_Date'] < group.iloc[i]['Start_Date']]
            hist_claims.append(past['Total_Claim_Count'].sum())
            premium = past['PrimaSuscrita'].sum()
            incurred = past['Total_Incurred_Amount'].sum()
            hist_ratios.append(incurred / premium if premium else None)
        group['Historical_Claim_Count'] = hist_claims
        group['Historical_Loss_Ratio'] = hist_ratios
        claim_count_results.append(group)

    enriched = pd.concat(claim_count_results, ignore_index=True)
    enriched['Current_Loss_Ratio'] = enriched['Total_Incurred_Amount'] / enriched['PrimaSuscrita']

    final_result = final_result.drop(columns=['PU_Key'], errors='ignore')
    final_result = final_result.merge(
        enriched[['Policy', 'Unit', 'Start_Date', 'Historical_Claim_Count', 'Historical_Loss_Ratio', 'Current_Loss_Ratio']],
        on=['Policy', 'Unit', 'Start_Date'],
        how='left'
    )

    print("🧼 Final formatting for export...")
    for col in tqdm(final_result.columns, desc="Final format"):
        if final_result[col].dtype == "object":
            final_result[col] = final_result[col].fillna("").astype(str)
        elif pd.api.types.is_numeric_dtype(final_result[col]):
            final_result[col] = pd.to_numeric(final_result[col], errors='coerce').fillna(0)

    # Export
    print("📅 Exporting final results...")
    final_result.to_csv('Insurance_1to1_Merged_With_Loss_Ratio.csv', index=False, encoding='utf-8-sig')
    final_result.to_parquet("Insurance_1to1_Merged_With_Loss_Ratio.parquet", index=False)

    # Optional: Remove accents
    def remove_accents(text):
        if isinstance(text, str):
            text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
            text = text.replace('ñ', 'n').replace('Ñ', 'N')
        return text

    print("🌐 Removing accents from string columns...")
    for col in tqdm(final_result.columns, desc="Removing accents"):
        if final_result[col].dtype == "object":
            final_result[col] = final_result[col].apply(remove_accents)

    final_result.to_csv('Insurance_1to1_Merged_With_Loss_Ratio_CleanText.csv', index=False, encoding='utf-8-sig')
else:
    final_result = pd.DataFrame()
    print("⚠️ No matched claims found.")

# Export unmatched claims
unmatched_all = unmatched_claims + [
    {
        'Policy': row['Policy'],
        'Unit': row['Unit'],
        'Start_Date': row['Start_Date'],
        'End_Date': row['End_Date'],
        'Reason': 'Policy-Unit not found in claims data'
    }
    for _, row in non_matching_rows.iterrows()
]
if unmatched_all:
    unmatched_df = pd.DataFrame(unmatched_all)
    unmatched_df.to_csv('Unmatched_Claims_Final.csv', index=False, encoding='utf-8-sig')
    print(f"⚠️ Exported {len(unmatched_df)} unmatched entries.")
else:
    print("✅ All insurance rows matched successfully.")

# Summary
print(f"\n💰 Total Incurred Matched: {final_result['Total_Incurred_Amount'].sum():,.2f}")
print(f"🕒 Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


/Users/ramonbatista/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading datasets...


/var/folders/91/12yv_3h95dg19ffkc2pk_ffm0000gn/T/ipykernel_4740/3312236332.py:11: DtypeWarning: Columns (23,24,25,26,27,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  insurance_data = pd.read_csv('Transformed_Auto_Insurance_Cleaned_For_GLM_V2.csv', encoding='utf-8-sig', dtype={'Policy': str, 'Unit': str})


🧹 Removing duplicate insurance rows...
📅 Converting date fields...
🧼 Formatting insurance_data...


Formatting insurance_data: 100%|██████████| 76/76 [00:02<00:00, 36.91it/s]


🔎 Filtering relevant policy-units...
📌 Matching claims to insurance rows (all that match)...


Matching multiple claims:  72%|███████▏  | 138434/191127 [28:41<13:31, 64.95it/s]  